<a href="https://colab.research.google.com/github/massinoLight/tp_AlgoBD/blob/main/Tp_Foret_Aleatoire_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 40 kB/s 
     |████████████████████████████████| 198 kB 69.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=424cc180b9b33c3ca0fb42c29626f36d4824dcba89225277c0769332fda8314e
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


##charger le fichier 

In [2]:
import requests 
file_url = "https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_libsvm_data.txt"
	
r = requests.get(file_url, stream = True) 

with open("/content/sample_data/sample_libsvm_data.txt", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 

##Context spark

In [3]:
from pyspark import SparkContext
# $example on$
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

sc = SparkContext(appName="PythonRandomForestClassification")

##Application du modéle sur le jeu de données (nb class=2,nombre d'arbre=3,max profondeur=4,gini)

In [4]:
data = MLUtils.loadLibSVMFile(sc, '/content/sample_data/sample_libsvm_data.txt')

(trainingData, testData) = data.randomSplit([0.7, 0.3])

   
model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                         numTrees=3, featureSubsetStrategy="auto",
                                         impurity='gini', maxDepth=4, maxBins=32)


predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
        lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification forest model:')
print(model.toDebugString())

    

Test Error = 0.0625
Learned classification forest model:
TreeEnsembleModel classifier with 3 trees

  Tree 0:
    If (feature 317 <= 8.0)
     If (feature 527 <= 18.5)
      If (feature 348 <= 128.0)
       Predict: 1.0
      Else (feature 348 > 128.0)
       Predict: 0.0
     Else (feature 527 > 18.5)
      Predict: 0.0
    Else (feature 317 > 8.0)
     If (feature 316 <= 253.5)
      If (feature 435 <= 39.0)
       Predict: 0.0
      Else (feature 435 > 39.0)
       Predict: 1.0
     Else (feature 316 > 253.5)
      Predict: 1.0
  Tree 1:
    If (feature 469 <= 4.0)
     If (feature 285 <= 7.5)
      If (feature 456 <= 31.5)
       Predict: 1.0
      Else (feature 456 > 31.5)
       Predict: 0.0
     Else (feature 285 > 7.5)
      Predict: 0.0
    Else (feature 469 > 4.0)
     Predict: 0.0
  Tree 2:
    If (feature 290 <= 14.5)
     If (feature 606 <= 41.5)
      Predict: 1.0
     Else (feature 606 > 41.5)
      Predict: 0.0
    Else (feature 290 > 14.5)
     Predict: 0.0



##Sauvegarde du modéle entrainé 

In [5]:
# Save and load model
model.save(sc, "model/RF/myRandomForestClassificationModel")
sameModel = RandomForestModel.load(sc, "model/RF/myRandomForestClassificationModel")